In [17]:
!python -m spacy download pt_core_news_sm


     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
     ---------------- ----------------------- 5.2/13.0 MB 29.0 MB/s eta 0:00:01
     ------------------------- -------------- 8.4/13.0 MB 21.7 MB/s eta 0:00:01
     ------------------------------------- - 12.3/13.0 MB 21.5 MB/s eta 0:00:01
     --------------------------------------- 13.0/13.0 MB 17.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import pandas as pd
import numpy as np
import re
import unicodedata
import nltk
import string
import spacy
import syllapy
from datetime import datetime
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet


In [20]:
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\miche/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\miche/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\miche/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\miche/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\miche/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [21]:
# Lista de arquivos CSV
file_paths = [
    '../raw/itens/itens-parte1.csv',
    '../raw/itens/itens-parte2.csv',
    '../raw/itens/itens-parte3.csv'
]

# Lista para armazenar os DataFrames de cada parte
dataframes = []

# Loop para carregar cada arquivo CSV e adicioná-lo à lista
for file_path in file_paths:
    df = pd.read_csv(file_path)
    dataframes.append(df)

# Concatenar todos os DataFrames em um único
df = pd.concat(dataframes, ignore_index=True)

# Exibir as primeiras linhas do DataFrame combinado
print(df.head())


                                   page  \
0  13db0ab1-eea2-4603-84c4-f40a876c7400   
1  92907b73-5cd3-4184-8d8c-e206aed2bf1c   
2  61e07f64-cddf-46f2-b50c-ea0a39c22050   
3  30e2e6c5-554a-48ed-a35f-6c6691c8ac9b   
4  9dff71eb-b681-40c7-ac8d-68017ac36675   

                                                 url  \
0  http://g1.globo.com/am/amazonas/noticia/2022/0...   
1  http://g1.globo.com/pa/santarem-regiao/noticia...   
2  http://g1.globo.com/mundo/noticia/2022/07/08/e...   
3  http://g1.globo.com/politica/noticia/2021/09/0...   
4  http://g1.globo.com/politica/noticia/2021/09/1...   

                      issued                   modified  \
0  2022-06-18 20:37:45+00:00  2023-04-15 00:02:08+00:00   
1  2019-06-20 17:19:52+00:00  2023-06-16 20:19:15+00:00   
2  2022-07-08 08:55:52+00:00  2023-04-15 04:25:39+00:00   
3  2021-09-09 19:06:46+00:00  2023-06-07 17:44:54+00:00   
4  2021-09-15 19:16:13+00:00  2023-06-07 17:43:39+00:00   

                                               ti

In [22]:
df = df.drop(columns=['modified'])

# Remover linhas com valores ausentes nas colunas essenciais
df = df.dropna(subset=['url', 'title', 'body'])

# Convertendo as colunas 'issued' para datetime
df['issued'] = pd.to_datetime(df['issued'], errors='coerce')

print(df.head())


                                   page  \
0  13db0ab1-eea2-4603-84c4-f40a876c7400   
1  92907b73-5cd3-4184-8d8c-e206aed2bf1c   
2  61e07f64-cddf-46f2-b50c-ea0a39c22050   
3  30e2e6c5-554a-48ed-a35f-6c6691c8ac9b   
4  9dff71eb-b681-40c7-ac8d-68017ac36675   

                                                 url  \
0  http://g1.globo.com/am/amazonas/noticia/2022/0...   
1  http://g1.globo.com/pa/santarem-regiao/noticia...   
2  http://g1.globo.com/mundo/noticia/2022/07/08/e...   
3  http://g1.globo.com/politica/noticia/2021/09/0...   
4  http://g1.globo.com/politica/noticia/2021/09/1...   

                     issued  \
0 2022-06-18 20:37:45+00:00   
1 2019-06-20 17:19:52+00:00   
2 2022-07-08 08:55:52+00:00   
3 2021-09-09 19:06:46+00:00   
4 2021-09-15 19:16:13+00:00   

                                               title  \
0  Caso Bruno e Dom: 3º suspeito tem prisão tempo...   
1  Linguajar dos santarenos é diferenciado e chei...   
2  Ex-premiê Shinzo Abe morre após ser baleado no

In [23]:
# Colunas que serão limpas
columns_to_clean = ["title", "body", "caption"]

# Verificando se as colunas existem no DataFrame
print("Colunas encontradas no DataFrame:")
for column in columns_to_clean:
    if column in df.columns:
        print(f"- {column} está presente")
    else:
        print(f"- {column} NÃO está presente")


Colunas encontradas no DataFrame:
- title está presente
- body está presente
- caption está presente


In [24]:
df = df.applymap(lambda x: x.replace('\n', ' ').replace('\r', ' ') if isinstance(x, str) else x)
print(df.head())

C:\Users\miche\AppData\Local\Temp\ipykernel_34216\1292987479.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('\n', ' ').replace('\r', ' ') if isinstance(x, str) else x)


                                   page  \
0  13db0ab1-eea2-4603-84c4-f40a876c7400   
1  92907b73-5cd3-4184-8d8c-e206aed2bf1c   
2  61e07f64-cddf-46f2-b50c-ea0a39c22050   
3  30e2e6c5-554a-48ed-a35f-6c6691c8ac9b   
4  9dff71eb-b681-40c7-ac8d-68017ac36675   

                                                 url  \
0  http://g1.globo.com/am/amazonas/noticia/2022/0...   
1  http://g1.globo.com/pa/santarem-regiao/noticia...   
2  http://g1.globo.com/mundo/noticia/2022/07/08/e...   
3  http://g1.globo.com/politica/noticia/2021/09/0...   
4  http://g1.globo.com/politica/noticia/2021/09/1...   

                     issued  \
0 2022-06-18 20:37:45+00:00   
1 2019-06-20 17:19:52+00:00   
2 2022-07-08 08:55:52+00:00   
3 2021-09-09 19:06:46+00:00   
4 2021-09-15 19:16:13+00:00   

                                               title  \
0  Caso Bruno e Dom: 3º suspeito tem prisão tempo...   
1  Linguajar dos santarenos é diferenciado e chei...   
2  Ex-premiê Shinzo Abe morre após ser baleado no

In [25]:
# Transformando os textos em minúsculas
for column in columns_to_clean:
    if column in df.columns:
        df[column] = df[column].str.lower()

# Exibindo as primeiras linhas após a transformação
print("Texto transformado para minúsculas:")
print(df.head())


Texto transformado para minúsculas:
                                   page  \
0  13db0ab1-eea2-4603-84c4-f40a876c7400   
1  92907b73-5cd3-4184-8d8c-e206aed2bf1c   
2  61e07f64-cddf-46f2-b50c-ea0a39c22050   
3  30e2e6c5-554a-48ed-a35f-6c6691c8ac9b   
4  9dff71eb-b681-40c7-ac8d-68017ac36675   

                                                 url  \
0  http://g1.globo.com/am/amazonas/noticia/2022/0...   
1  http://g1.globo.com/pa/santarem-regiao/noticia...   
2  http://g1.globo.com/mundo/noticia/2022/07/08/e...   
3  http://g1.globo.com/politica/noticia/2021/09/0...   
4  http://g1.globo.com/politica/noticia/2021/09/1...   

                     issued  \
0 2022-06-18 20:37:45+00:00   
1 2019-06-20 17:19:52+00:00   
2 2022-07-08 08:55:52+00:00   
3 2021-09-09 19:06:46+00:00   
4 2021-09-15 19:16:13+00:00   

                                               title  \
0  caso bruno e dom: 3º suspeito tem prisão tempo...   
1  linguajar dos santarenos é diferenciado e chei...   
2  ex-premiê 

In [26]:
def normalize_unicode(text):
    if isinstance(text, str):
        return unicodedata.normalize("NFKC", text)  # Normaliza caracteres Unicode
    return text

# Aplicando normalização Unicode antes da remoção de acentos
for column in ["title", "body", "caption"]:
    if column in df.columns:
        df[column] = df[column].apply(normalize_unicode)  # Normalizar caracteres Unicode

# Exibir os dados após a limpeza
print(df[["title", "body", "caption"]].head(10))

                                               title  \
0  caso bruno e dom: 3o suspeito tem prisão tempo...   
1  linguajar dos santarenos é diferenciado e chei...   
2  ex-premiê shinzo abe morre após ser baleado no...   
3  relator no stf, fachin vota contra marco tempo...   
4   após 2 votos, pedido de vista suspende julgam...   
5  o que é o marco temporal sobre terras indígena...   
6  shannen doherty, de 'barrados no baile', compa...   
7  como cornershop criou hit 'complicado' com mis...   
8                                       equipe do g1   
9  terceiro suspeito de participar dos assassinat...   

                                                body  \
0  após audiência de custódia, a justiça do amazo...   
1  vista aérea de santarém ádrio denner/ ad produ...   
2  novo vídeo mostra que assassino de shinzo abe ...   
3  relator no stf, fachin vota contra marco tempo...   
4  após um pedido de vista (mais tempo para análi...   
5  marco temporal sobre terras indígenas: enten

In [27]:
# 🔹 Função para remover caracteres de escape como "\r" e "\n"
def remove_escape_chars(text):
    return text.replace("\r", " ").replace("\n", " ") if isinstance(text, str) else text

# 🔹 Função para remover expressões regulares específicas
def remove_regex_patterns(text, patterns):
    if isinstance(text, str):
        for pattern in patterns:
            text = re.sub(pattern, '', text)
    return text

# 🔹 Função para remover números
def remove_numbers(text):
    return re.sub(r'\d+', '', text) if isinstance(text, str) else text

# 🔹 Função para remover pontuação (mantendo acentuação)
def remove_punctuation(text):
    # Remover pontuação, mas manter caracteres acentuados (á, à, ã, etc.)
    return re.sub(r'[^\w\sáàãâäéèêëíìîïóòôõöúùûüç]', '', text) if isinstance(text, str) else text

# 🔹 Função para remover espaços extras
def remove_extra_spaces(text):
    return ' '.join(text.split()) if isinstance(text, str) else text

# 🔹 Função para substituir hífens por espaços
def replace_hyphens(text):
    return re.sub(r'(?<=\w)-(?=\w)', ' ', text) if isinstance(text, str) else text

# 🔹 Função para remover colchetes e seu conteúdo
def remove_brackets(text):
    return re.sub(r'\[.*?\]', '', text) if isinstance(text, str) else text

# 🔹 Função para remover palavras vazias
def remove_empty_words(text):
    return ' '.join([word for word in text.split() if word != '']) if isinstance(text, str) else text

# 🔹 Função para remover underscores
def remove_underscores(text):
    return text.replace("_", "") if isinstance(text, str) else text

# 🔹 Padrões regex a serem removidos
regex_patterns = [
    r"http\S+",   # URLs
    r"@\w+",      # Menções
    r"#\w+",      # Hashtags
    r"º",         # Caractere º
    r"[^a-zA-Z0-9\sáàãâäéèêëíìîïóòôõöúùûüç]",  # Outros caracteres especiais (exceto espaços e caracteres acentuados)
]

# Aplicando as funções de pré-processamento nas colunas
for column in columns_to_clean:
    if column in df.columns:
        df[column] = df[column].apply(remove_escape_chars)    # Remover caracteres como \r e \n
        df[column] = df[column].apply(replace_hyphens)        # Substituir hífens por espaços
        df[column] = df[column].apply(lambda x: remove_regex_patterns(x, regex_patterns))  # Remover padrões regex
        df[column] = df[column].apply(remove_numbers)         # Remover números
        df[column] = df[column].apply(remove_punctuation)     # Remover pontuação
        df[column] = df[column].apply(remove_extra_spaces)    # Remover espaços extras
        df[column] = df[column].apply(remove_brackets)        # Remover colchetes
        df[column] = df[column].apply(remove_empty_words)     # Remover palavras vazias
        df[column] = df[column].apply(remove_underscores)     # Remover underscores

# Exibir as primeiras linhas após o pré-processamento
print(df.head())


                                   page  \
0  13db0ab1-eea2-4603-84c4-f40a876c7400   
1  92907b73-5cd3-4184-8d8c-e206aed2bf1c   
2  61e07f64-cddf-46f2-b50c-ea0a39c22050   
3  30e2e6c5-554a-48ed-a35f-6c6691c8ac9b   
4  9dff71eb-b681-40c7-ac8d-68017ac36675   

                                                 url  \
0  http://g1.globo.com/am/amazonas/noticia/2022/0...   
1  http://g1.globo.com/pa/santarem-regiao/noticia...   
2  http://g1.globo.com/mundo/noticia/2022/07/08/e...   
3  http://g1.globo.com/politica/noticia/2021/09/0...   
4  http://g1.globo.com/politica/noticia/2021/09/1...   

                     issued  \
0 2022-06-18 20:37:45+00:00   
1 2019-06-20 17:19:52+00:00   
2 2022-07-08 08:55:52+00:00   
3 2021-09-09 19:06:46+00:00   
4 2021-09-15 19:16:13+00:00   

                                               title  \
0  caso bruno e dom o suspeito tem prisão temporá...   
1  linguajar dos santarenos é diferenciado e chei...   
2  ex premiê shinzo abe morre após ser baleado no

In [28]:
# Função para remover emojis
def remove_emojis(text):
    """
    Remove emojis de um texto utilizando expressões regulares.
    :param text: Texto de entrada.
    :return: Texto sem emojis.
    """
    if isinstance(text, str):  # Verifica se o valor é uma string
        emoji_pattern = re.compile(
            "[\U0001F600-\U0001F64F"  # Emoticons
            "\U0001F300-\U0001F5FF"  # Símbolos e pictogramas
            "\U0001F680-\U0001F6FF"  # Transportes e símbolos adicionais
            "\U0001F1E0-\U0001F1FF"  # Bandeiras (sub-regiões)
            "\U00002500-\U00002BEF"  # Outros símbolos
            "\U00002700-\U000027BF"  # Dingbats
            "\U0001F900-\U0001F9FF"  # Suplementos de emojis
            "\U0001FA70-\U0001FAFF"  # Suplementos adicionais
            "\U00002600-\U000026FF"  # Diversos símbolos (como ♪)
            "\U0001F000-\U0001F02F"  # Cartas de jogo
            "]+",
            flags=re.UNICODE,
        )
        return emoji_pattern.sub(r"", text)  # Remove emojis
    return text  # Retorna o valor original se não for uma string

for column in columns_to_clean:
    if column in df.columns:
        df[column] = df[column].apply(remove_emojis)

# Exibindo as primeiras linhas para verificar
print(df.head())

                                   page  \
0  13db0ab1-eea2-4603-84c4-f40a876c7400   
1  92907b73-5cd3-4184-8d8c-e206aed2bf1c   
2  61e07f64-cddf-46f2-b50c-ea0a39c22050   
3  30e2e6c5-554a-48ed-a35f-6c6691c8ac9b   
4  9dff71eb-b681-40c7-ac8d-68017ac36675   

                                                 url  \
0  http://g1.globo.com/am/amazonas/noticia/2022/0...   
1  http://g1.globo.com/pa/santarem-regiao/noticia...   
2  http://g1.globo.com/mundo/noticia/2022/07/08/e...   
3  http://g1.globo.com/politica/noticia/2021/09/0...   
4  http://g1.globo.com/politica/noticia/2021/09/1...   

                     issued  \
0 2022-06-18 20:37:45+00:00   
1 2019-06-20 17:19:52+00:00   
2 2022-07-08 08:55:52+00:00   
3 2021-09-09 19:06:46+00:00   
4 2021-09-15 19:16:13+00:00   

                                               title  \
0  caso bruno e dom o suspeito tem prisão temporá...   
1  linguajar dos santarenos é diferenciado e chei...   
2  ex premiê shinzo abe morre após ser baleado no

In [29]:
# Função para tokenizar o texto
def tokenize_text(text):
    return word_tokenize(text)  # Retorna uma lista de tokens

# Função para remover stopwords de uma lista de tokens
def remove_stopwords(tokens, language='portuguese'):
    stop_words = set(stopwords.words(language))
    return [word for word in tokens if word.lower() not in stop_words]  # Remove stopwords e converte para minúsculas

# Aplicando as funções no DataFrame corretamente
for column in columns_to_clean:
    if column in df.columns:
        df[column] = df[column].apply(lambda x: remove_stopwords(tokenize_text(str(x))))
# Exibir as primeiras linhas do DataFrame após a limpeza
print(df.head())

                                   page  \
0  13db0ab1-eea2-4603-84c4-f40a876c7400   
1  92907b73-5cd3-4184-8d8c-e206aed2bf1c   
2  61e07f64-cddf-46f2-b50c-ea0a39c22050   
3  30e2e6c5-554a-48ed-a35f-6c6691c8ac9b   
4  9dff71eb-b681-40c7-ac8d-68017ac36675   

                                                 url  \
0  http://g1.globo.com/am/amazonas/noticia/2022/0...   
1  http://g1.globo.com/pa/santarem-regiao/noticia...   
2  http://g1.globo.com/mundo/noticia/2022/07/08/e...   
3  http://g1.globo.com/politica/noticia/2021/09/0...   
4  http://g1.globo.com/politica/noticia/2021/09/1...   

                     issued  \
0 2022-06-18 20:37:45+00:00   
1 2019-06-20 17:19:52+00:00   
2 2022-07-08 08:55:52+00:00   
3 2021-09-09 19:06:46+00:00   
4 2021-09-15 19:16:13+00:00   

                                               title  \
0  [caso, bruno, dom, suspeito, prisão, temporári...   
1  [linguajar, santarenos, diferenciado, cheio, i...   
2  [ex, premiê, shinzo, abe, morre, após, baleado

In [30]:
nlp = spacy.load("pt_core_news_sm")

# Função para lematizar texto com POS usando Spacy
def lemmatize_tokens(text):
    # Processar o texto com Spacy para análise
    doc = nlp(text)  # O Spacy processa a string diretamente

    # Lematizando com base no POS e removendo as stopwords
    lemmatized_tokens = [token.lemma_ for token in doc if not token.is_stop]

    # Retornar os tokens lematizados como uma string
    return ' '.join(lemmatized_tokens)

# Aplicando a função em cada uma das colunas de texto
for column in columns_to_clean:
    if column in df.columns:
        # Garantir que o texto seja passado como string para a função de lematização
        df[column] = df[column].apply(lambda x: lemmatize_tokens(' '.join(x)))

# Exibir as primeiras linhas após o pré-processamento
print(df.head())


                                   page  \
0  13db0ab1-eea2-4603-84c4-f40a876c7400   
1  92907b73-5cd3-4184-8d8c-e206aed2bf1c   
2  61e07f64-cddf-46f2-b50c-ea0a39c22050   
3  30e2e6c5-554a-48ed-a35f-6c6691c8ac9b   
4  9dff71eb-b681-40c7-ac8d-68017ac36675   

                                                 url  \
0  http://g1.globo.com/am/amazonas/noticia/2022/0...   
1  http://g1.globo.com/pa/santarem-regiao/noticia...   
2  http://g1.globo.com/mundo/noticia/2022/07/08/e...   
3  http://g1.globo.com/politica/noticia/2021/09/0...   
4  http://g1.globo.com/politica/noticia/2021/09/1...   

                     issued  \
0 2022-06-18 20:37:45+00:00   
1 2019-06-20 17:19:52+00:00   
2 2022-07-08 08:55:52+00:00   
3 2021-09-09 19:06:46+00:00   
4 2021-09-15 19:16:13+00:00   

                                               title  \
0  caso bruno dom suspeito prisão temporário decr...   
1  linguajar santareno diferenciar cheio identida...   
2           ex premiê shinzo abe morrer balear Ja

In [31]:
def remove_person_names(text):
    # Processar o texto com Spacy para análise
    doc = nlp(text)
    # Criar uma lista com os tokens que não são entidades do tipo "pessoa"
    cleaned_text = [token.text for token in doc if token.ent_type_ != 'PER']

    # Unir os tokens novamente em uma string
    return ' '.join(cleaned_text)

for column in columns_to_clean:
    if column in df.columns:
        df[column] = df[column].apply(lambda x: remove_person_names(x))

# Exibir as primeiras linhas do DataFrame após a limpeza
print(df.head())

                                   page  \
0  13db0ab1-eea2-4603-84c4-f40a876c7400   
1  92907b73-5cd3-4184-8d8c-e206aed2bf1c   
2  61e07f64-cddf-46f2-b50c-ea0a39c22050   
3  30e2e6c5-554a-48ed-a35f-6c6691c8ac9b   
4  9dff71eb-b681-40c7-ac8d-68017ac36675   

                                                 url  \
0  http://g1.globo.com/am/amazonas/noticia/2022/0...   
1  http://g1.globo.com/pa/santarem-regiao/noticia...   
2  http://g1.globo.com/mundo/noticia/2022/07/08/e...   
3  http://g1.globo.com/politica/noticia/2021/09/0...   
4  http://g1.globo.com/politica/noticia/2021/09/1...   

                     issued  \
0 2022-06-18 20:37:45+00:00   
1 2019-06-20 17:19:52+00:00   
2 2022-07-08 08:55:52+00:00   
3 2021-09-09 19:06:46+00:00   
4 2021-09-15 19:16:13+00:00   

                                               title  \
0  caso bruno dom suspeito prisão temporário decr...   
1  linguajar santareno diferenciar cheio identida...   
2           ex premiê shinzo abe morrer balear Ja

In [36]:
# Função para remover palavras com uma ou duas letras
def remove_short_words(text):
    words = text.split()  # Divide o texto em palavras
    # Filtra palavras com mais de 2 letras
    filtered_words = [word for word in words if len(word) > 2]
    return ' '.join(filtered_words)

# Aplicar a função de limpeza em várias colunas
for column in columns_to_clean:
    if column in df.columns:
        df[column] = df[column].apply(remove_short_words)

# Exibir as primeiras linhas do DataFrame após a limpeza
print(df.head())

                                   page  \
0  13db0ab1-eea2-4603-84c4-f40a876c7400   
1  92907b73-5cd3-4184-8d8c-e206aed2bf1c   
2  61e07f64-cddf-46f2-b50c-ea0a39c22050   
3  30e2e6c5-554a-48ed-a35f-6c6691c8ac9b   
4  9dff71eb-b681-40c7-ac8d-68017ac36675   

                                                 url  \
0  http://g1.globo.com/am/amazonas/noticia/2022/0...   
1  http://g1.globo.com/pa/santarem-regiao/noticia...   
2  http://g1.globo.com/mundo/noticia/2022/07/08/e...   
3  http://g1.globo.com/politica/noticia/2021/09/0...   
4  http://g1.globo.com/politica/noticia/2021/09/1...   

                     issued  \
0 2022-06-18 20:37:45+00:00   
1 2019-06-20 17:19:52+00:00   
2 2022-07-08 08:55:52+00:00   
3 2021-09-09 19:06:46+00:00   
4 2021-09-15 19:16:13+00:00   

                                               title  \
0  caso bruno dom suspeito prisão temporário decr...   
1  linguajar santareno diferenciar cheio identida...   
2              premiê shinzo abe morrer balear Ja

In [37]:
# Salvar o DataFrame processado em um arquivo CSV
df.to_csv('../preprocessed/itens/dados_preprocessados.csv', index=False)

print("Arquivo CSV salvo com sucesso!")


Arquivo CSV salvo com sucesso!
